# Ouray Defensible Space Analysis

Author: **Bryce A Young** (git bryceayoung) | 
Created: **2024-12-05** | 
Modified: **2025-02-11**

In this notebook, we analyze fuel distributions within the defensible space of every building in Ouray County.

Documents for cleaning and preparing raw data for the analysis are located in this repo under `workflows/data_prep`.

#### Data 
- (vector) Microsoft Building Footprints
- (raster) LiDAR-derived rasters

#### Workflow 
- Create HIZ boundaries around each building footprint
- Count number of homes within each HIZ
- Get zonal summary values of each LiDAR-derived raster for each HIZ

## Step 0: Setup Environment
---

In [1]:
import os
### Directory ###
# Repository
os.chdir(r'D:/_PROJECTS/P001_OurayParcel/ouray')
# Root workspace
ws = r'D:/_PROJECTS/P001_OurayParcel'

### Data paths ###
# Folder where all the data inputs and outputs will live
data = os.path.join(ws, 'data')
# Folder for geoms (microsoft building footprints, parcels, and county boundary)
geoms = os.path.join(data, 'county_geoms')
# Folder containing LiDAR-derived rasters
rasters = os.path.join(data, 'tiffs_from_las')
# Scratch folder
scratch = os.path.join(data, '_temp')

# Ensure correct working directory
os.getcwd()

'D:\\_PROJECTS\\P001_OurayParcel\\ouray'

## Step 1: Create HIZ Boundaries around each Structure
---
**Background: obtaining building footprints**  
*I searched "Ouray County Colorado Microsoft Building Footprints" and found a Colorado state website that had parsed the Microsoft Building Footprints dataset into county datasets for all of Colorado. So I was able to directly download the Ouray footprints without having to do any data manipulation myself.*

*After downloading the footprints, I added them to ArcGIS Pro and viewed them on top of my LiDAR data. Especially the `zentropy` layer shows buildings very well. I noticed that the footprint geometries are offset from the location of the buildings in the LiDAR rasters. So I used the 'Transform' tool in ArcGIS Pro 3.4.0 to put control points and target points (links) with Rubbersheet (natural neighbor) as the transformation method. This operation brought the building footprints closer to the buildings as they appear in the Z-entropy LiDAR raster. This created 105 anchor points througout the county extent where buildings are present.*

*The resulting shapefile was saved to `buildings_rs_WKID26913.shp` where 'rs' denotes 'rubber sheet' transformation of the original footprint layer. Original footprints are saved in the same directory as `Ouray_County_Buildings.shp` and geopackage as `buildings_WKID26913.gpkg`.*

*In order to display my results, I randomized the building footprints and selected 9 random buildings which I plotted in an ArcGIS Pro layout with a 100m square buffer around the centroid. The maps in the layout show the pre-rubbersheet and post-rubbersheet building footprints on top of the Z-entropy LiDAR raster. This shows that the building footprints are well-aligned with the LiDAR rasters in order to ensure that the structures are not being considered in the LiDAR analysis.*

### Import geometries and run HIZ script
The function `simple_hiz` from `utils.HIZ` in this repo creates a single defensible space zone around each building footprint. Looking at the randomly selected buildings, 3m is a sufficient buffer. I will put this buffer in the parameters and say that the outer edge will be that buffer + 30m, so the outer edge will be 33m from the building footprint. 

30m is chosen because of the NFPA and Cohen definitions of defensible space that are supported by post fire assessments, such as the Lahaina post-fire investigation (Hedayati et al. 2024).

In [2]:
import geopandas as gpd

# Import building footprints
mbf = gpd.read_file(os.path.join(geoms, 'buildings_rs_WKID26913.shp'))
mbf.head()

,County,geometry
0,Ouray County,"POLYGON ((233065.876 4241049.368, 233062.927 4..."
1,Ouray County,"POLYGON ((242799.828 4243540.283, 242798.994 4..."
2,Ouray County,"POLYGON ((251221.424 4245433.145, 251222.638 4..."
3,Ouray County,"POLYGON ((244094.335 4241435.666, 244103.748 4..."
4,Ouray County,"POLYGON ((254081.763 4242776.159, 254083.688 4..."


In [3]:
from utils.HIZ import simple_hiz

# Create defensible space zone around all building footprints, saving as its own gdf
hiz = simple_hiz(mbf)
hiz.head() # Preview data

,geometry
0,"POLYGON ((233082.966 4241022.314, 233080.295 4..."
1,"POLYGON ((242805.059 4243571.853, 242808.129 4..."
2,"POLYGON ((251182.204 4245395.048, 251179.74 42..."
3,"POLYGON ((244062.393 4241433.743, 244062.359 4..."
4,"POLYGON ((254114.554 4242761.882, 254113.584 4..."


In [4]:
# Save to file
hiz.to_file(os.path.join(geoms, 'hiz_WKID26913.gpkg'), driver='GPKG')
print('hiz geom saved to file!')

hiz geom saved to file!


## Step 2: Structure Density
---

In order to obtain a proxy for structure density, we are going to count the number of adjacent structures in the vicinity of each home. We will also compute distance to nearest structure. We will append these counts to the HIZ geopackage.

In [5]:
# Count number of structures within defensible space
from utils.HIZ import structures_in_hiz

counts_df = structures_in_hiz(mbf, hiz)
counts_df.head()

,footprint_index,intersections
0,0,0
1,1,2
2,2,0
3,3,0
4,4,0


In [6]:
counts_df.shape

(4533, 2)

In [7]:
# Save df to csv in scratch folder
counts_df.to_csv(os.path.join(scratch, 'structure_count.csv'))

In [8]:
# Compute minimum structure separation distance per structure
from utils.HIZ import min_ssd

mbf = min_ssd(mbf)
mbf.head()

,County,geometry,min_ssd
0,Ouray County,"POLYGON ((233065.876 4241049.368, 233062.927 4...",221.938317
1,Ouray County,"POLYGON ((242799.828 4243540.283, 242798.994 4...",11.936200
2,Ouray County,"POLYGON ((251221.424 4245433.145, 251222.638 4...",213.672097
3,Ouray County,"POLYGON ((244094.335 4241435.666, 244103.748 4...",183.675975
4,Ouray County,"POLYGON ((254081.763 4242776.159, 254083.688 4...",101.386718


In [9]:
# Save mbf to file in scratch folder
mbf.to_file(os.path.join(scratch, 'mbf_wui_ssd.gpkg'), driver='GPKG')

## Step 3: Get canopy cover info for each HIZ
---

#### Read in LiDAR-derived rasters

I created these rasters in R using the **lidR** package and methods documented in this repository under `r_workflows`.

#### Get raster values per HIZ across the county

Recall the geodataframes of home ignition zones that we created at the beginning of this workflow. It contains geometries for **4533** building footprins and their home ignition zones. We're going to look at the vegetation within each HIZ to get an idea of the vegetation profile around each home.

For the purposes of reducing wildfire risk to individual homes, it's important that we analyze raster values within each buffer zone. GeoPandas comes in handy for this task.

Our workflow for this calculation - written in plain English - is as follows:
1. **Mask raster with geometries**: Recall that each home ignition zone gdf is a geoseries of Shapely geometries. We need to iterate through **each** of these **4533** objects and use them to mask our raster, object by object.
2. **Calculate average raster values within each HIZ**: once the mask is applied to only include a single building buffer, we'll take the average of all the pixels in the area. 
3. **Store respective canopy cover value per HIZ as a new column in the HIZ dataframe:** The average values will be stored as columns and each value will correspond to the geometry in the same row.

In [2]:
import geopandas as gpd
hiz = gpd.read_file(os.path.join(geoms, 'hiz_WKID26913.gpkg'))

In [4]:
hiz.head()

,geometry
0,"POLYGON ((233082.966 4241022.314, 233080.295 4..."
1,"POLYGON ((242805.059 4243571.853, 242808.129 4..."
2,"POLYGON ((251182.204 4245395.048, 251179.74 42..."
3,"POLYGON ((244062.393 4241433.743, 244062.359 4..."
4,"POLYGON ((254114.554 4242761.882, 254113.584 4..."


In [3]:
from utils.HIZ import raster_stats

# File naming conventions, starting with aoi (a county, community, etc.) and ending with spatial reference
wkid = 'WKID26913'
aoi = 'ouray'

# Define names of rasters for dynamic column naming in gdf output
rnames = ['cc0_2m', 'cc_2_4m', 'cc4_8m', 'cc8_40m']
raster_paths = [os.path.join(rasters, f'{aoi}_cc0_2m_{wkid}.tif'),
                os.path.join(rasters, f'{aoi}_cc2_4m_{wkid}.tif'),
                os.path.join(rasters, f'{aoi}_cc4_8m_{wkid}.tif'),
                os.path.join(rasters, f'{aoi}_cc8_40m_{wkid}.tif')
                ]

hiz_full = raster_stats(hiz, raster_paths, rnames, stats=['mean'])
hiz_full.head()

KeyError: 'footprint_index'

#### Append values to HIZ geopackages

In [ ]:
# Code to append values

In [3]:
# Save updated GDFs

## Discussion and Conclusions
---

- Broader picture of wildfire risk to communities and susceptibility and defensible space
- Describe the project in its entirety, in brief
- Here's how this notebook/workflow fits into the larger project
- Here's what we did.
- Here were the most difficult parts.
- Here were the key parts to get right.
- Time estimates for running the code.
- Information on computational expense.
- Room for improvement

## NOTES: Possibble to use this code somewhere?
---

We're going to clip our rasters to the extent of the given geometry for each of these.

Our simplified proxy for NFPA compliance is going to be the **average max height** of vegetation pixels within each zone.

The process of clipping rasters to geometries requires a fe steps. First, we're going to mask the max height raster `arr_max` with each HIZ geometry - `z1`, `z2`, and `z3`. To do this, we have to rasterize each geometry. The following 3 code blocks accomplish that, and save the masked raster outputs to arrays.

In [83]:
from rasterio.mask import mask
from rasterio.io import MemoryFile

# Prepare geometries from GeoDataFrame for masking
geoms = z1['geometry'].values # Zone 1

# Create a new in-memory dataset
with MemoryFile() as memfile:
    # Define metadata based on the properties of arr_max
    meta = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'count': 1,
        'width': arr_max.shape[1],
        'height': arr_max.shape[0],
        'crs': hmax.crs,  # Update with the correct CRS as necessary
        'transform': hmax.transform,  # Update with the correct transform if available
        'nodata': np.nan
    }

    with memfile.open(**meta) as hmax_dataset:
        # Write arr_max to the in-memory dataset
        hmax_dataset.write(arr_max, 1)

        # Apply the mask using the geometries
        z1_out_values, z1_out_transform = mask(hmax_dataset, geoms, crop=True, nodata=np.nan)


In [84]:
geoms = z2['geometry'].values # Zone 2

# Create a new in-memory dataset
with MemoryFile() as memfile:
    # Define metadata based on the properties of arr_max
    meta = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'count': 1,
        'width': arr_max.shape[1],
        'height': arr_max.shape[0],
        'crs': hmax.crs,  # Update with the correct CRS as necessary
        'transform': hmax.transform,  # Update with the correct transform if available
        'nodata': np.nan
    }

    with memfile.open(**meta) as hmax_dataset:
        # Write arr_max to the in-memory dataset
        hmax_dataset.write(arr_max, 1)

        # Apply the mask using the geometries
        z2_out_values, z2_out_transform = mask(hmax_dataset, geoms, crop=True, nodata=np.nan)


In [85]:
geoms = z3['geometry'].values # Zone 3

# Create a new in-memory dataset
with MemoryFile() as memfile:
    # Define metadata based on the properties of arr_max
    meta = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'count': 1,
        'width': arr_max.shape[1],
        'height': arr_max.shape[0],
        'crs': hmax.crs,  # Update with the correct CRS as necessary
        'transform': hmax.transform,  # Update with the correct transform if available
        'nodata': np.nan
    }

    with memfile.open(**meta) as hmax_dataset:
        # Write arr_max to the in-memory dataset
        hmax_dataset.write(arr_max, 1)

        # Apply the mask using the geometries
        z3_out_values, z3_out_transform = mask(hmax_dataset, geoms, crop=True, nodata=np.nan)


Next, let's define a function for computing the average value of all the pixels.

We know that our zones are all donut-shaped. If we divide the sum of the pixels by the size of the array, then pixels both inside and outside the donut will be included in the size, and that will throw off our calculation. The above 3 code blocks have set all the values outside and inside of the donuts to `NaN`, so now we make sure our `calculate_avg` function divided by the number of *non-NaN* pixels in the array.

### RESULTS
All that's left to do is calculate the average max height per zone. Here are the results:

In [87]:
calculate_avg(z3_out_values)

2.3484971919686135

In [88]:
calculate_avg(z2_out_values)

3.054104777520576

In [89]:
calculate_avg(z1_out_values)

3.9157764165088382

### CONCLUSIONS

There are some insights and some caveats to draw from this from a fire mitigation perspective.

For a NFPA-compliant home ignition zone, we would expect to see taller average height values further away from the home. Instead, we are seeing the reverse.

The Z1 values have possibly been skewed by the home itself. Since the Microsoft building footprint does not perfectly align with the location of the building in the point cloud, it is likely that the building itself has been included in the max height calculation. A way to handle this would be to classify the point cloud to include buildings, and derive the building footprint directly from the LiDAR data.

The Z2 and Z3 values are probably more reliable. It tells us that there is tall vegetation surrounding the home. If the average height exceeds 3m in Z2, it is likely that this property could benefit from removing some trees and ensuring that the ground is clear of continuous flammable fuels. Since Z3 average max height is lower, it's likely that Z3 will not carry fire, although the average max height could be skewed by clumps of tall, continuous forest fuels interspersed with grassland, which is typical of this environment.

My conclusion is that average max height does not produce an actionable insight for home risk reduction. There is potential to develop better ways of producing actionable insights for home wildfire risk mitigation, but none have been developed (based on a meticulous review of over 150 papers and models). While the results presented here may prove to be a useful predictor in a random forest or multi-layer perceptron model, I will continue to test the value of different LiDAR metrics using the framework developed through this project.